In [30]:
import os
import dotenv
dotenv.load_dotenv()
import google.generativeai as genai
import typing_extensions as typing
import json

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
quiz_generation_config = {
  "temperature": 0.7,
  "top_p": 0.95,
  "top_k": 20,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
}

summary_generation_config = {
  "temperature": 0.7,
  "top_p": 0.95,
  "top_k": 60,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
GEMINI_PRO_2_5 = "gemini-2.5-flash-preview-05-20"

quiz_model = genai.GenerativeModel(
  model_name=GEMINI_PRO_2_5,
  generation_config=quiz_generation_config,
  system_instruction="You are a helpful assistant which helps teachers generate quiz from given content based on user requirements.",
)
summary_model = genai.GenerativeModel(
  model_name=GEMINI_PRO_2_5,
  generation_config=summary_generation_config,
  system_instruction="You are a helpful assistant which helps teachers generate summary from given content based on user requirements.",
)

quiz_chat_session = quiz_model.start_chat(
  history=[
  ]
)
summary_chat_session = summary_model.start_chat(
  history=[
  ]
)



In [3]:
import nest_asyncio
nest_asyncio.apply()

# bring in deps
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# set up parser
parser = LlamaParse(
    result_type="markdown"  # "markdown" and "text" are available
)

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}

# Ensure the async method is awaited
import asyncio

# Define a global variable to store the text

async def load_documents():
    global all_text
    global documents
    documents = await SimpleDirectoryReader(input_files=['/Users/aftab/ClassTest/TheHistoryofComputers1.pdf'], file_extractor=file_extractor).aload_data(show_progress=True)
    all_text = "\n".join([doc.text_resource.text for doc in documents])

# Run the async function
asyncio.run(load_documents())

  0%|          | 0/1 [00:00<?, ?it/s]

Started parsing the file under job_id 69096d42-dcd9-4a1d-beb0-b40406692e07


100%|██████████| 1/1 [00:28<00:00, 28.31s/it]


In [ ]:
quiz_format = """
{
  quiz_title:str,
    questions: [
      {
      "question_id": int,
      "question": str,
      "options": [{"option_id": int, "option": str}],
      "correct_option_id": int
      }
    ]
}
"""

In [31]:
def generate_summary(text: str) -> str:
    response = summary_chat_session.send_message(f"Generate datailed and precise summary in points without leaving any small detail from the given content: {text}")


    return response.text

In [257]:
MAX_QUESTIONS = 20
def generate_quiz(text, num_questions=MAX_QUESTIONS):
    response = quiz_chat_session.send_message(f"Create a quiz of {num_questions} questions returning data in this JSON format: \n{quiz_format} on the content given below\n\n{text}")
    return response.text

In [10]:
answers = []
answers.append(generate_quiz(all_text,15))


In [11]:

formatted_json = []
for answer in answers:
  formatted_json.append(json.loads(answer))
print(formatted_json)

[{'quiz_title': 'History of Computers Quiz', 'questions': [{'question_id': 1, 'question': 'What was the first computing device?', 'options': [{'option_id': 1, 'option': 'Astrolabe'}, {'option_id': 2, 'option': 'Abacus'}, {'option_id': 3, 'option': 'Pascaline'}, {'option_id': 4, 'option': 'Difference Engine'}], 'correct_option_id': 2}, {'question_id': 2, 'question': 'Who invented the first digital computer that could only add?', 'options': [{'option_id': 1, 'option': 'Gottfried Wilhelm von Leibniz'}, {'option_id': 2, 'option': 'Charles Babbage'}, {'option_id': 3, 'option': 'Blaise Pascal'}, {'option_id': 4, 'option': 'Alan Turing'}], 'correct_option_id': 3}, {'question_id': 3, 'question': 'Which inventor created a computer that could add and multiply?', 'options': [{'option_id': 1, 'option': 'Blaise Pascal'}, {'option_id': 2, 'option': 'Charles Babbage'}, {'option_id': 3, 'option': 'Gottfried Wilhelm von Leibniz'}, {'option_id': 4, 'option': 'John Napier'}], 'correct_option_id': 3}, {'q

In [32]:
summary = generate_summary(all_text)

In [ ]:

with open(f"quiz.json", "w") as f:
  json.dump(json.loads(formatted_json), f)